# Install and import required libraries

In [ ]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import google.colab
from tqdm import tqdm
import re
import warnings
import csv

from flair.models import SequenceTagger
from collections import Counter
from flair.datasets import UD_INDONESIAN
from flair.embeddings import WordEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.data import Sentence

warnings.filterwarnings("ignore")

# Import data

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
df_tweets = pd.read_csv('/content/drive/MyDrive/TA/tweets/ruhutsitompul/done_labelled_ruhutsitompul.csv', delimiter=',', error_bad_lines=False)
df_tweets

,Unnamed: 0,ID,Tweet Time,Convo ID,Likes Count,Retweets Count,Tweets,Tweets(rendered),in_reply_to_tweet_id,in_reply_to,...,clean_tweets_len,Month,social,historical,dehumanization,accusation,attack,loyalty,threat,Tweets.1
0,0,1585739644011892736,2022-10-27 21:06:07,1585739644011892736,460,80,Sama banget Komentar ini dgn para pendukung se...,Sama banget Komentar ini dgn para pendukung se...,NaN,NaN,...,11,October,1,0,0,0,0.0,0,0,Sama banget Komentar ini dgn para pendukung se...
1,1,1585737538806853632,2022-10-27 20:57:45,1585737538806853632,420,92,Ha ha ha sigundul penguasa ancol karena selama...,Ha ha ha sigundul penguasa ancol karena selama...,NaN,NaN,...,15,October,1,0,0,1,0.0,0,0,Ha ha ha sigundul penguasa ancol karena selama...
2,2,1584780616369049605,2022-10-25 05:35:17,1584780616369049605,343,40,Kasihan Ibu ini jadi korban akibat dicuci otak...,Kasihan Ibu ini jadi korban akibat dicuci otak...,NaN,NaN,...,14,October,0,0,1,1,0.0,0,0,Kasihan Ibu ini jadi korban akibat dicuci otak...
3,3,1584778048120836096,2022-10-25 05:25:05,1584778048120836096,212,23,Ha ha hakadrun pada stresssssss mengenai beber...,Ha ha hakadrun pada stresssssss mengenai beber...,NaN,NaN,...,18,October,1,0,1,0,0.0,0,0,Ha ha hakadrun pada stresssssss mengenai beber...
4,4,1584774254502912000,2022-10-25 05:10:00,1584774254502912000,860,63,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...,NaN,NaN,...,33,October,0,0,1,0,0.0,0,0,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4178,4178,1118666888647602177,2019-04-18 00:05:44,1118666888647602177,0,0,@NediSetiadi Kami sedih krn biar sudah kalian ...,@NediSetiadi Kami sedih krn biar sudah kalian ...,NaN,NaN,...,14,April,1,0,0,1,0.0,0,0,@NediSetiadi Kami sedih krn biar sudah kalian ...
4179,4179,1118665503642247170,2019-04-18 00:00:14,1118665503642247170,7,0,@TanYoana haiya Amoy ini makin stresssssss kac...,@TanYoana haiya Amoy ini makin stresssssss kac...,NaN,NaN,...,5,April,0,0,0,0,0.0,0,0,@TanYoana haiya Amoy ini makin stresssssss kac...
4180,4180,1118664923301572608,2019-04-17 23:57:56,1118664923301572608,1684,200,Terima kasih KPU BAWASLU DKPP sbg Penyelenggar...,Terima kasih KPU BAWASLU DKPP sbg Penyelenggar...,NaN,NaN,...,31,April,0,0,0,0,0.0,0,0,Terima kasih KPU BAWASLU DKPP sbg Penyelenggar...
4181,4181,1118396091148460032,2019-04-17 06:09:41,1118396091148460032,6,0,"@panca66 Ha ha ha maaf memilih diri sendiri, s...","@panca66 Ha ha ha maaf memilih diri sendiri, s...",NaN,NaN,...,5,April,0,0,0,0,0.0,0,0,"@panca66 Ha ha ha maaf memilih diri sendiri, s..."


In [ ]:
# Remove tweets without any aspect
df_tweets = df_tweets.loc[(df_tweets[['social', 'historical', 'dehumanization', 'accusation', 'attack', 'loyalty', 'threat']] != 0).any(axis=1)]
df_tweets = df_tweets.reset_index(drop=True)
df_tweets

,Unnamed: 0,ID,Tweet Time,Convo ID,Likes Count,Retweets Count,Tweets,Tweets(rendered),in_reply_to_tweet_id,in_reply_to,...,clean_tweets_len,Month,social,historical,dehumanization,accusation,attack,loyalty,threat,Tweets.1
0,0,1585739644011892736,2022-10-27 21:06:07,1585739644011892736,460,80,Sama banget Komentar ini dgn para pendukung se...,Sama banget Komentar ini dgn para pendukung se...,NaN,NaN,...,11,October,1,0,0,0,0.0,0,0,Sama banget Komentar ini dgn para pendukung se...
1,1,1585737538806853632,2022-10-27 20:57:45,1585737538806853632,420,92,Ha ha ha sigundul penguasa ancol karena selama...,Ha ha ha sigundul penguasa ancol karena selama...,NaN,NaN,...,15,October,1,0,0,1,0.0,0,0,Ha ha ha sigundul penguasa ancol karena selama...
2,2,1584780616369049605,2022-10-25 05:35:17,1584780616369049605,343,40,Kasihan Ibu ini jadi korban akibat dicuci otak...,Kasihan Ibu ini jadi korban akibat dicuci otak...,NaN,NaN,...,14,October,0,0,1,1,0.0,0,0,Kasihan Ibu ini jadi korban akibat dicuci otak...
3,3,1584778048120836096,2022-10-25 05:25:05,1584778048120836096,212,23,Ha ha hakadrun pada stresssssss mengenai beber...,Ha ha hakadrun pada stresssssss mengenai beber...,NaN,NaN,...,18,October,1,0,1,0,0.0,0,0,Ha ha hakadrun pada stresssssss mengenai beber...
4,4,1584774254502912000,2022-10-25 05:10:00,1584774254502912000,860,63,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...,NaN,NaN,...,33,October,0,0,1,0,0.0,0,0,Kok sewot dgn Pidato Sambutan Bpk Joko Widodo ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1659,4151,1120875995962101760,2019-04-24 02:23:56,1120875995962101760,26,4,"@ustadtengkuzul Belajarlah menerima kekalahan,...","@ustadtengkuzul Belajarlah menerima kekalahan,...",NaN,NaN,...,12,April,1,0,0,1,0.0,0,0,"@ustadtengkuzul Belajarlah menerima kekalahan,..."
1660,4153,1120574450410217472,2019-04-23 06:25:42,1120574450410217472,1660,264,Aneh tapi Nyata ini bukan HOAX di Uk...,Aneh tapi Nyata ini bukan HOAX di Uk...,NaN,NaN,...,13,April,0,0,0,1,0.0,0,0,Aneh tapi Nyata ini bukan HOAX di Uk...
1661,4166,1119465028040527872,2019-04-20 04:57:15,1119465028040527872,1311,193,"Tim Sukses Pak Jokowi bukan Panglima Hukum, Pr...","Tim Sukses Pak Jokowi bukan Panglima Hukum, Pr...",NaN,NaN,...,24,April,1,0,0,1,0.0,0,0,"Tim Sukses Pak Jokowi bukan Panglima Hukum, Pr..."
1662,4168,1119429279811428353,2019-04-20 02:35:12,1119429279811428353,10,0,@FerdinandHutah2 Masih saja kau menggonggong s...,@FerdinandHutah2 Masih saja kau menggonggong s...,NaN,NaN,...,6,April,0,0,1,0,0.0,0,0,@FerdinandHutah2 Masih saja kau menggonggong s...


# POS tagging with flair

In [ ]:
# 1. get the corpus
corpus = UD_INDONESIAN().downsample(0.1)
print(corpus)

# 2. what label do we want to predict?
label_type = 'upos'

# 3. make the label dictionary from the corpus
label_dict = corpus.make_label_dictionary(label_type=label_type)
print(label_dict)

# 4. initialize embeddings
embedding_types = [

    WordEmbeddings('id-crawl'),
    WordEmbeddings('id'),

    # comment in this line to use character embeddings
    # CharacterEmbeddings(),

    # comment in these lines to use flair embeddings
    # FlairEmbeddings('news-forward'),
    # FlairEmbeddings('news-backward'),
]

embeddings = StackedEmbeddings(embeddings=embedding_types)

# 5. initialize sequence tagger
tagger = SequenceTagger(hidden_size=256,
                        embeddings=embeddings,
                        tag_dictionary=label_dict,
                        tag_type=label_type,
                        use_crf=True)

# 6. initialize trainer
trainer = ModelTrainer(tagger, corpus)

# 7. start training
trainer.train('resources/taggers/example-upos',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=10)

2023-03-13 06:20:55,991 Reading data from /root/.flair/datasets/ud_indonesian
2023-03-13 06:20:55,995 Train: /root/.flair/datasets/ud_indonesian/id_gsd-ud-train.conllu
2023-03-13 06:20:55,999 Dev: /root/.flair/datasets/ud_indonesian/id_gsd-ud-dev.conllu
2023-03-13 06:20:56,001 Test: /root/.flair/datasets/ud_indonesian/id_gsd-ud-test.conllu
2023-03-13 06:21:09,877 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-03-13 06:21:10,712 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2023-03-13 06:21:12,333 Warning: An empty Sentence was created! Are there empty strings in your dataset?
Corpus: 448 train + 56 dev + 56 test sentences
2023-03-13 06:21:12,340 Computing label dictionary. Progress:


448it [00:00, 13588.92it/s]

2023-03-13 06:21:12,419 Dictionary created for label 'upos' with 17 values: NOUN (seen 2158 times), PROPN (seen 1832 times), PUNCT (seen 1470 times), VERB (seen 970 times), ADP (seen 882 times), PRON (seen 486 times), NUM (seen 408 times), ADJ (seen 377 times), CCONJ (seen 305 times), DET (seen 297 times), ADV (seen 279 times), AUX (seen 179 times), SCONJ (seen 158 times), PART (seen 72 times), SYM (seen 14 times), X (seen 7 times)
Dictionary with 17 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, NUM, ADJ, CCONJ, DET, ADV, AUX, SCONJ, PART, SYM, X


2023-03-13 06:21:36,582 SequenceTagger predicts: Dictionary with 17 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, NUM, ADJ, CCONJ, DET, ADV, AUX, SCONJ, PART, SYM, X
2023-03-13 06:21:36,621 ----------------------------------------------------------------------------------------------------
2023-03-13 06:21:36,624 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'id-crawl'
      (embedding): Embedding(1000000, 300)
    )
    (list_embedding_1): WordEmbeddings(
      'id'
      (embedding): Embedding(300686, 300)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=600, out_features=600, bias=True)
  (rnn): LSTM(600, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=19, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)"
2023-03-13 06:21:36,625 --------------------------------------------------------------

100%|██████████| 2/2 [00:00<00:00,  2.33it/s]

...

2023-03-13 06:25:01,838 ----------------------------------------------------------------------------------------------------
2023-03-13 06:25:06,297 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, NUM, ADJ, CCONJ, DET, ADV, AUX, SCONJ, PART, SYM, X, <START>, <STOP>


100%|██████████| 2/2 [00:00<00:00,  5.71it/s]

2023-03-13 06:25:07,317 Evaluating as a multi-label problem: False


2023-03-13 06:25:07,335 0.7908	0.7908	0.7908	0.7908
2023-03-13 06:25:07,337 
Results:
- F-score (micro) 0.7908
- F-score (macro) 0.6244
- Accuracy 0.7908

By class:
              precision    recall  f1-score   support

        NOUN     0.7462    0.8778    0.8067       221
       PROPN     0.9493    0.7616    0.8452       172
        VERB     0.7471    0.9549    0.8383       133
       PUNCT     1.0000    1.0000    1.0000       137
         ADP     0.9167    0.8381    0.8756       105
        PRON     0.7952    0.9296    0.8571        71
         ADV     0.2222    0.3125    0.2597        32
       CCONJ     0.9062    0.8788    0.8923        33
         DET     0.5926    0.4444    0.5079        36
         ADJ     0.4737    0.2093    0.2903        43
         NUM     0.8621    0.7576    0.8065        33
         AUX     0.4545    0.2000    0.2778        25
       SCONJ     0.2941    0.3125    0.3030        16
        PART     0.5000    0.1111    0.1818         9

    accuracy           

{'test_score': 0.7908067542213884,
 'dev_score_history': [0.23976608187134502,
  0.43859649122807015,
  0.5519005847953217,
  0.5935672514619883,
  0.6549707602339181,
  0.6980994152046783,
  0.7331871345029239,
  0.7609649122807017,
  0.7814327485380117,
  0.7997076023391813],
 'train_loss_history': [2.819584507399529,
  2.1270101627051434,
  1.7136281053779436,
  1.4805878098863579,
  1.3175628458535475,
  1.1915952899563498,
  1.079175219237985,
  0.9923594397488377,
  0.9025038948005104,
  0.8330865524165529],
 'dev_loss_history': [2.3777313232421875,
  1.8053864240646362,
  1.4546347856521606,
  1.2840070724487305,
  1.1046547889709473,
  0.9594604969024658,
  0.8435547351837158,
  0.7774122953414917,
  0.6908206939697266,
  0.6244803071022034]}

#2-gram

In [ ]:
# load the Flair POS tagger for Indonesian language
tagger = SequenceTagger.load('resources/taggers/example-upos/best-model.pt')

# ambil subset data hanya untuk tweet dengan kategori 1
social_tweets = df_tweets[df_tweets["social"] == 1]["clean_tweets"].tolist()

# cari pola tag POS bigram paling umum
patterns = []
for tweet in social_tweets:
    s = Sentence(tweet)
    tagger.predict(s)
    tags = [token.get_labels()[0].value for token in s]
    bigrams = zip(tags, tags[1:])
    for bigram in bigrams:
        pattern = " ".join(bigram)
        patterns.append(pattern)

counter = Counter(patterns)
most_common = counter.most_common(10)

# mencetak hasil
print("Top 10 most frequent POS tag bigram patterns in social aspect (Flair, Indonesian model):")
for i, (pattern, count) in enumerate(most_common):
    print(f"{i+1}. {pattern} ({count} occurrences)")

2023-03-13 06:25:10,884 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, NUM, ADJ, CCONJ, DET, ADV, AUX, SCONJ, PART, SYM, X, <START>, <STOP>
Top 10 most frequent POS tag bigram patterns in social aspect (Flair, Indonesian model):
1. NOUN NOUN (2472 occurrences)
2. PROPN PROPN (1663 occurrences)
3. VERB NOUN (1407 occurrences)
4. PROPN VERB (924 occurrences)
5. NOUN PROPN (695 occurrences)
6. NOUN VERB (400 occurrences)
7. NOUN ADJ (340 occurrences)
8. ADJ NOUN (229 occurrences)
9. VERB VERB (181 occurrences)
10. VERB PROPN (163 occurrences)


In [ ]:
# load the Flair POS tagger for Indonesian language
tagger = SequenceTagger.load('resources/taggers/example-upos/best-model.pt')

# ambil subset data hanya untuk tweet dengan kategori 1
historical_tweets = df_tweets[df_tweets["historical"] == 1]["clean_tweets"].tolist()

# cari pola tag POS bigram paling umum
patterns = []
for tweet in historical_tweets:
    s = Sentence(tweet)
    tagger.predict(s)
    tags = [token.get_labels()[0].value for token in s]
    bigrams = zip(tags, tags[1:])
    for bigram in bigrams:
        pattern = " ".join(bigram)
        patterns.append(pattern)

counter = Counter(patterns)
most_common = counter.most_common(10)

# mencetak hasil
print("Top 10 most frequent POS tag bigram patterns in historical aspect (Flair, Indonesian model):")
for i, (pattern, count) in enumerate(most_common):
    print(f"{i+1}. {pattern} ({count} occurrences)")

2023-03-13 06:25:22,460 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, NUM, ADJ, CCONJ, DET, ADV, AUX, SCONJ, PART, SYM, X, <START>, <STOP>
Top 10 most frequent POS tag bigram patterns in historical aspect (Flair, Indonesian model):
1. NOUN NOUN (122 occurrences)
2. PROPN PROPN (88 occurrences)
3. VERB NOUN (63 occurrences)
4. PROPN VERB (49 occurrences)
5. NOUN PROPN (47 occurrences)
6. NOUN VERB (13 occurrences)
7. NOUN ADJ (9 occurrences)
8. VERB VERB (8 occurrences)
9. NOUN PRON (7 occurrences)
10. PROPN PUNCT (7 occurrences)


In [ ]:
# load the Flair POS tagger for Indonesian language
tagger = SequenceTagger.load('resources/taggers/example-upos/best-model.pt')

# ambil subset data hanya untuk tweet dengan kategori 1
dehumanization_tweets = df_tweets[df_tweets["dehumanization"] == 1]["clean_tweets"].tolist()

# cari pola tag POS bigram paling umum
patterns = []
for tweet in dehumanization_tweets:
    s = Sentence(tweet)
    tagger.predict(s)
    tags = [token.get_labels()[0].value for token in s]
    bigrams = zip(tags, tags[1:])
    for bigram in bigrams:
        pattern = " ".join(bigram)
        patterns.append(pattern)

counter = Counter(patterns)
most_common = counter.most_common(10)

# mencetak hasil
print("Top 10 most frequent POS tag bigram patterns in dehumanization aspect (Flair, Indonesian model):")
for i, (pattern, count) in enumerate(most_common):
    print(f"{i+1}. {pattern} ({count} occurrences)")

2023-03-13 06:25:26,741 SequenceTagger predicts: Dictionary with 19 tags: <unk>, NOUN, PROPN, PUNCT, VERB, ADP, PRON, NUM, ADJ, CCONJ, DET, ADV, AUX, SCONJ, PART, SYM, X, <START>, <STOP>
Top 10 most frequent POS tag bigram patterns in dehumanization aspect (Flair, Indonesian model):
1. NOUN NOUN (1987 occurrences)
2. PROPN PROPN (1490 occurrences)
3. VERB NOUN (1045 occurrences)
4. PROPN VERB (686 occurrences)
5. NOUN PROPN (563 occurrences)
6. NOUN ADJ (342 occurrences)
7. NOUN VERB (285 occurrences)
8. NOUN PRON (256 occurrences)
9. PROPN PUNCT (212 occurrences)
10. ADJ NOUN (172 occurrences)
